In [1]:
import numpy as np
import pandas as pd
import unidecode

# Criando o DataFrame geral
### Carregando os dados principais:

In [2]:
casos_df = pd.read_csv('datasets/dados_casos.csv')
df_pib = pd.read_excel('datasets/PIB_2010-2019.xls')
vax_df = pd.read_csv('datasets/dados_vacinacao.csv')
dem_df = pd.read_csv('datasets/den_demografica-PB2010.csv')

### Começando a criação do general_df

Uma vez que a maioria dos dados utilizados são do DataFrame de casos, inicialzaremos a variávl general_df recebendo o DataFrame casos_df.

In [3]:
general_df = casos_df

Para evitar erros, as strings correspondentes às cidades serão convertidas para Unidecode, sem acentos ou cedilhas, e maiúsculas. Essa correção será feita em todos os DataFrames.

In [4]:
general_df.city = general_df.city.apply(unidecode.unidecode).str.upper().str.strip()
general_df = general_df.drop(columns= ['date', 'state', 'place_type', 'is_last', 'city_ibge_code'])
general_df.head(5)

,city,confirmed,deaths,estimated_population,confirmed_per_100k_inhabitants,death_rate
0,AGUA BRANCA,1128,14,10306,10945.08054,0.0124
1,AGUIAR,528,5,5630,9378.33037,0.0095
2,ALAGOA GRANDE,4117,72,28439,14476.59904,0.0175
3,ALAGOA NOVA,1511,25,20921,7222.40811,0.0165
4,ALAGOINHA,2072,25,14560,14230.76923,0.0121


## Adicionando o PIB ao DataFrame:

Dados sobre o PIB de todo o país:

In [5]:
df_pib

,Ano,Código da Grande Região,Nome da Grande Região,Código da Unidade da Federação,Sigla da Unidade da Federação,Nome da Unidade da Federação,Código do Município,Nome do Município,Região Metropolitana,Código da Mesorregião,...,"Valor adicionado bruto da Indústria,\na preços correntes\n(R$ 1.000)","Valor adicionado bruto dos Serviços,\na preços correntes \n- exceto Administração, defesa, educação e saúde públicas e seguridade social\n(R$ 1.000)","Valor adicionado bruto da Administração, defesa, educação e saúde públicas e seguridade social, \na preços correntes\n(R$ 1.000)","Valor adicionado bruto total, \na preços correntes\n(R$ 1.000)","Impostos, líquidos de subsídios, sobre produtos, \na preços correntes\n(R$ 1.000)","Produto Interno Bruto, \na preços correntes\n(R$ 1.000)","Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)",Atividade com maior valor adicionado bruto,Atividade com segundo maior valor adicionado bruto,Atividade com terceiro maior valor adicionado bruto
0,2010,1,Norte,11,RO,Rondônia,1100015,Alta Floresta D'Oeste,NaN,1102,...,16118.534,6.249618e+04,9.324466e+04,2.411198e+05,2.095711e+04,2.620769e+05,10731.18,"Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços
1,2010,1,Norte,11,RO,Rondônia,1100023,Ariquemes,NaN,1102,...,287138.585,4.949463e+05,3.438677e+05,1.199664e+06,1.650296e+05,1.364694e+06,15103.86,"Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...
2,2010,1,Norte,11,RO,Rondônia,1100031,Cabixi,NaN,1102,...,3252.506,1.267721e+04,2.517024e+04,6.540077e+04,4.210342e+03,6.961111e+04,11033.62,"Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços
3,2010,1,Norte,11,RO,Rondônia,1100049,Cacoal,NaN,1102,...,182051.537,4.654473e+05,2.984543e+05,1.041212e+06,1.452817e+05,1.186494e+06,15095.15,"Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...
4,2010,1,Norte,11,RO,Rondônia,1100056,Cerejeiras,NaN,1102,...,19734.484,8.072499e+04,6.301827e+04,1.924542e+05,2.956703e+04,2.220212e+05,13037.06,"Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55680,2019,5,Centro-oeste,52,GO,Goiás,5222005,Vianópolis,NaN,5205,...,26348.404,1.750361e+05,7.031241e+04,4.459215e+05,3.774179e+04,4.836633e+05,34888.79,Demais serviços,"Agricultura, inclusive apoio à agricultura e a...","Administração, defesa, educação e saúde públic..."
55681,2019,5,Centro-oeste,52,GO,Goiás,5222054,Vicentinópolis,NaN,5205,...,47140.720,1.171335e+05,4.416814e+04,3.087939e+05,3.356172e+04,3.423556e+05,39157.68,"Agricultura, inclusive apoio à agricultura e a...",Demais serviços,"Administração, defesa, educação e saúde públic..."
55682,2019,5,Centro-oeste,52,GO,Goiás,5222203,Vila Boa,Região Integrada de Desenvolvimento do Distrit...,5204,...,16186.594,3.314555e+04,3.181999e+04,1.052628e+05,8.405675e+03,1.136685e+05,18419.78,"Administração, defesa, educação e saúde públic...",Demais serviços,"Agricultura, inclusive apoio à agricultura e a..."
55683,2019,5,Centro-oeste,52,GO,Goiás,5222302,Vila Propício,Região Integrada de Desenvolvimento do Distrit...,5204,...,29867.460,3.531253e+04,2.931617e+04,1.990382e+05,7.933047e+03,2.069712e+05,35555.96,"Agricultura, inclusive apoio à agricultura e a...",Demais serviços,"Administração, defesa, educação e saúde públic..."


Tendo em vista que as análises do relatório serão restritas à Paraíba e há maior interesse em dados mais novos, a variável pb_pib_df será criada recebendo apenas esses dados. Além disso, a variável será restrita ao nome do município e o seu ib per capita associado.

In [6]:
pb_pib_df = df_pib[(df_pib['Sigla da Unidade da Federação'] == 'PB') & (df_pib['Ano'] == 2019)]

pb_pib_df = pb_pib_df.loc[:, ['Nome do Município', 'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)']]

pb_pib_df.columns = ['city', 'pib_per_capita']

pb_pib_df.reset_index(drop = True, inplace = True)

In [7]:
pb_pib_df.city = pb_pib_df.city.apply(unidecode.unidecode).str.upper().str.strip()
pb_pib_df

,city,pib_per_capita
0,AGUA BRANCA,8074.16
1,AGUIAR,8897.54
2,ALAGOA GRANDE,9699.38
3,ALAGOA NOVA,15631.55
4,ALAGOINHA,7934.57
...,...,...
218,UIRAUNA,11427.53
219,UMBUZEIRO,7738.11
220,VARZEA,11215.80
221,VIEIROPOLIS,7827.71


In [8]:
general_df = general_df.set_index('city').join(pb_pib_df.set_index('city'))
general_df.head(2)

,confirmed,deaths,estimated_population,confirmed_per_100k_inhabitants,death_rate,pib_per_capita
city,,,,,,
AGUA BRANCA,1128,14,10306,10945.08054,0.0124,8074.16
AGUIAR,528,5,5630,9378.33037,0.0095,8897.54


## Adicionando o número de vacinados ao DataFrame:

Para descobrir o número de vacinados por mnicípio, será feito um value_counts em cima do estabelecimento que vacinou cada município.

In [9]:
vax_df.head(3)

,paciente_idade,paciente_dataNascimento,paciente_enumSexoBiologico,paciente_racaCor_codigo,paciente_racaCor_valor,paciente_endereco_nmMunicipio,paciente_endereco_cep,estabelecimento_municipio_nome,vacina_grupoAtendimento_codigo,vacina_grupoAtendimento_nome,vacina_categoria_codigo,vacina_categoria_nome,vacina_dataAplicacao,vacina_descricao_dose,vacina_codigo,vacina_nome,paciente_id
0,44,1977-04-18,M,1,BRANCA,ARAPIRACA,57510,PATOS,912,Médico,9,Trabalhadores de Saúde,2021-11-11,Reforço,87,COVID-19 PFIZER - COMIRNATY,f9495cb04daf5adb495d22c9c88b31883f191fb762ab27...
1,5,2016-11-18,M,1,BRANCA,PARELHAS,None,CAMPINA GRANDE,207,Pessoas de 5 a 11 anos,0,NaN,2022-02-07,1ª Dose,99,COVID-19 PEDIÁTRICA - PFIZER COMIRNATY,52288faace8c986872faca7101868ae8a23fefd435cd51...
2,39,1982-01-23,F,99,SEM INFORMACAO,CAMPINA GRANDE,58410,CAMPINA GRANDE,201,Pessoas de 18 a 64 anos,2,Faixa Etária,2021-07-16,1ª Dose,85,COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,0ffaed327aac7fe8abf0b35471b0b1fa60f9fc48ff7d9c...


In [10]:
count_df = pd.DataFrame(vax_df['estabelecimento_municipio_nome'].value_counts()).sort_index()
count_df.reset_index(inplace= True)
count_df.columns = ['city', 'n_vaccinations']
count_df

,city,n_vaccinations
0,AGUA BRANCA,6754
1,AGUIAR,3790
2,ALAGOA GRANDE,21931
3,ALAGOA NOVA,14518
4,ALAGOINHA,9389
...,...,...
218,UMBUZEIRO,6807
219,VARZEA,2524
220,VIEIROPOLIS,3012
221,VISTA SERRANA,2526


In [11]:
general_df = general_df.join(count_df.set_index('city'))

In [12]:
general_df

,confirmed,deaths,estimated_population,confirmed_per_100k_inhabitants,death_rate,pib_per_capita,n_vaccinations
city,,,,,,,
AGUA BRANCA,1128,14,10306,10945.08054,0.0124,8074.16,6754
AGUIAR,528,5,5630,9378.33037,0.0095,8897.54,3790
ALAGOA GRANDE,4117,72,28439,14476.59904,0.0175,9699.38,21931
ALAGOA NOVA,1511,25,20921,7222.40811,0.0165,15631.55,14518
ALAGOINHA,2072,25,14560,14230.76923,0.0121,7934.57,9389
...,...,...,...,...,...,...,...
UMBUZEIRO,545,14,9911,5498.94057,0.0257,7738.11,6807
VARZEA,132,1,2841,4646.25132,0.0076,11215.80,2524
VIEIROPOLIS,288,5,5372,5361.13179,0.0174,7827.71,3012


## Adicionando a densidade demográfica ao DataFrame:

Para adicionar a densidade demográfica, bastou padrozinar o gráfico e juntar ao DataFrame principal.

In [13]:
dem_df.cidade = dem_df.cidade.apply(unidecode.unidecode).str.upper().str.strip()

In [19]:
dem_df

,city,demographic_density
0,AGUA BRANCA,39.94
1,AGUIAR,16.04
2,ALAGOA GRANDE,88.84
3,ALAGOA NOVA,160.98
4,ALAGOINHA,139.99
...,...,...
218,UMBUZEIRO,51.28
219,VARZEA,13.15
220,VIEIROPOLIS,34.37
221,VISTA SERRANA,57.24


In [14]:
dem_df.columns = ['city', 'demographic_density']
dem_df

,city,demographic_density
0,AGUA BRANCA,39.94
1,AGUIAR,16.04
2,ALAGOA GRANDE,88.84
3,ALAGOA NOVA,160.98
4,ALAGOINHA,139.99
...,...,...
218,UMBUZEIRO,51.28
219,VARZEA,13.15
220,VIEIROPOLIS,34.37
221,VISTA SERRANA,57.24


In [15]:
general_df =general_df.join(dem_df.set_index('city'))

In [16]:
general_df.reset_index(inplace= True)

In [17]:
general_df.head(2)

,city,confirmed,deaths,estimated_population,confirmed_per_100k_inhabitants,death_rate,pib_per_capita,n_vaccinations,demographic_density
0,AGUA BRANCA,1128,14,10306,10945.08054,0.0124,8074.16,6754,39.94
1,AGUIAR,528,5,5630,9378.33037,0.0095,8897.54,3790,16.04


## Salvando o DataSet:

Feitas as alterações, adicionamos os dados a um csv na pasta de DataSets:

In [18]:
general_df.to_csv('datasets/dados_pb.csv', index= False)